# One model - Without having into account the categories

In [46]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
    
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import KFold

In [2]:
outcomesDf = pd.read_csv('./outcomes_clean.tsv',sep='\t')

In [3]:
x_column_names = ["retail","bidincrement","bidfee","flg_click_only","flg_beginnerauction","flg_fixedprice","flg_endprice"]

In [20]:
X = outcomesDf[x_column_names]
y = outcomesDf["price"]

In [47]:
kFoldMedianAbsoluteErrorsRandomForestRegressor = []
kFoldMedianAbsoluteErrorsKNeighborsRegressor = []
kFoldMedianAbsoluteErrorsDecisionTreeRegressor = []
kFoldMedianAbsoluteErrorsLinearRegression = []

kf = KFold(n_splits=5, random_state=1)
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #RandomForestRegressor
    model=RandomForestRegressor(random_state=1)
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsRandomForestRegressor.append(kFoldMedianAbsoluteError)

    #KNeighborsRegressor
    model = KNeighborsRegressor()
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsKNeighborsRegressor.append(kFoldMedianAbsoluteError)
    
    #DecisionTreeRegressor
    model = DecisionTreeRegressor()
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsDecisionTreeRegressor.append(kFoldMedianAbsoluteError)
    
    #LinearRegression
    model = LinearRegression()
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsLinearRegression.append(kFoldMedianAbsoluteError)    
    
print(np.mean(kFoldMedianAbsoluteErrorsRandomForestRegressor))
print(np.mean(kFoldMedianAbsoluteErrorsKNeighborsRegressor))
print(np.mean(kFoldMedianAbsoluteErrorsDecisionTreeRegressor))
print(np.mean(kFoldMedianAbsoluteErrorsLinearRegression))

11.1752458141
11.9772
11.0470921819
24.2493912238


# One Model - Having into account the categories

In [61]:
outcomesDf = pd.read_csv('./outcomes_clean.tsv',sep='\t')

In [ ]:
ver si esto tiene la resolucion correcta

In [62]:
productDescriptionToVectorDf = pd.read_excel('./productDescriptionToVector.xlsx')

productDescriptionToVector={}
for item in productDescriptionToVectorDf.columns:
    productDescriptionToVector[item] = productDescriptionToVectorDf[item]

In [73]:
error: los clusters se tienen que hacer en el train

SyntaxError: invalid syntax (<ipython-input-73-0f44d29f1c3a>, line 1)

In [63]:
productVectors = list(productDescriptionToVector.values())

from sklearn.cluster import KMeans
km = KMeans(n_clusters=15)
km.fit(productVectors)
clusters = km.labels_.tolist()
clusteredCategoriesDf = pd.DataFrame({'desc': list(productDescriptionToVector.keys()), 'cat_cluster': clusters})

In [64]:
print(len(clusteredCategoriesDf['cat_cluster'].unique()))
for cluster in clusteredCategoriesDf['cat_cluster'].unique():
    print(clusteredCategoriesDf[clusteredCategoriesDf['cat_cluster']==cluster])

15
      cat_cluster                                               desc
0               6         LG  26.5 Cu. Ft. Side by Side Refrigerator
12              6                          1 Ounce Gold Bar (31.10g)
14              6                                   10 Gram Gold Bar
17              6         12' x 30" Family Size Metal Frame Pool Set
25              6   3-Feet Carolina Artificial Prelit Christmas Tree
26              6                                   30 Ounces Silver
31              6                           5 g Gold Bar (0.16 t oz)
36              6               8-Piece Stainless-Steel Cookware Set
64              6                      AccuSharp 001 Knife Sharpener
99              6    All-Clad Stainless All-Purpose Steamer with Lid
100             6   All-Clad Stainless-Steel 6-1/2-Quart Slow Cooker
103             6                          Amco Double Digital Timer
105             6      American Weigh Precision Digital Pocket Scale
150             6            As

      cat_cluster                                              desc
178             0  Beats by Dr. Dre Tour High Resolution Headphones
692             0       JBuds Hi-Fi Noise-Reducing Ear Buds (Black)
985             0  Monster Turbine High Performance In-Ear Speakers
1378            0                Sennheiser CX300-B Earbuds (Black)
1379            0             Sennheiser HD-212 Headphones (Silver)
1380            0        Sennheiser Pc121 In-Ear Mono-Aural Headset
1462            0                  Sony MDR-XD100 Stereo Headphones
      cat_cluster                                               desc
221            12               Boss Femme Perfume for Women 3pc Set
222            12                Boss Selection By Hugo Boss For Men
232            12                 Braun 5885 Contour Electric Shaver
233            12         Braun PocketGo 370 Battery Operated Shaver
234            12             Braun Series 3 340 (4775) Men's Shaver
235            12                    Braun

In [65]:
categories = clusteredCategoriesDf['cat_cluster']
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
categories
categories = categories.values.reshape(len(categories), 1)
categories
onehot_encoded = onehot_encoder.fit_transform(categories)
onehot_encoded

ncolumns = np.shape(onehot_encoded)[1]
clusterColumNames = ['cluster_']*ncolumns
for clusterNumber in np.arange(ncolumns):
    clusterColumNames[clusterNumber] += str(clusterNumber)

clusteredCategoriesDfOneHotEnc = pd.DataFrame(onehot_encoded,columns=clusterColumNames)
clusteredCategoriesDfOneHotEnc

,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [66]:
cluster_column_names = clusteredCategoriesDfOneHotEnc.columns.values
clusteredCategoriesDfOneHotEnc['desc'] = clusteredCategoriesDf['desc']

In [68]:
outcomesDfWithCatOneHot = outcomesDf.merge(clusteredCategoriesDfOneHotEnc,how='left')

In [69]:
normal_x_column_names = ["retail","bidincrement","bidfee","flg_click_only","flg_beginnerauction","flg_fixedprice","flg_endprice"]
column_names_with_clusters = np.concatenate([normal_x_column_names,cluster_column_names])

In [70]:
X = outcomesDfWithCatOneHot[column_names_with_clusters]
y = outcomesDfWithCatOneHot["price"]

In [71]:
kFoldMedianAbsoluteErrorsRandomForestRegressor = []
kFoldMedianAbsoluteErrorsKNeighborsRegressor = []
kFoldMedianAbsoluteErrorsDecisionTreeRegressor = []
kFoldMedianAbsoluteErrorsLinearRegression = []

kf = KFold(n_splits=5, random_state=1)
for train_index, test_index in kf.split(X):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #RandomForestRegressor
    model=RandomForestRegressor(random_state=1)
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsRandomForestRegressor.append(kFoldMedianAbsoluteError)

    #KNeighborsRegressor
    model = KNeighborsRegressor()
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsKNeighborsRegressor.append(kFoldMedianAbsoluteError)
    
    #DecisionTreeRegressor
    model = DecisionTreeRegressor()
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsDecisionTreeRegressor.append(kFoldMedianAbsoluteError)
    
    #LinearRegression
    model = LinearRegression()
    model.fit(X_train,y_train)
    P_price = model.predict(X_test)
    
    kFoldMedianAbsoluteError = median_absolute_error(y_test,P_price)
    kFoldMedianAbsoluteErrorsLinearRegression.append(kFoldMedianAbsoluteError)    
    
print(np.mean(kFoldMedianAbsoluteErrorsRandomForestRegressor))
print(np.mean(kFoldMedianAbsoluteErrorsKNeighborsRegressor))
print(np.mean(kFoldMedianAbsoluteErrorsDecisionTreeRegressor))
print(np.mean(kFoldMedianAbsoluteErrorsLinearRegression))

10.3892604558
11.054
10.4496695021
22.185679063


# Multiple models by category